In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier
from cfmining.action_set import ActionSet

from results import save_result

Using Python-MIP package version 1.7.2


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [3]:
giveme_df = pd.read_csv('data/student/student-mat.csv', sep=';', index_col=None).dropna()
y = 1*(giveme_df['G3']>=10)
X = giveme_df.drop(['G1', 'G2', 'G3'], axis=1)

In [4]:
X = giveme_df.drop(['G1', 'G2', 'G3'], axis=1)
X['school'] = X['school'].map({'GP': 0, 'MS': 1})
X['sex'] = X['sex'].map({'M': 0, 'F': 1})
X['address'] = X['address'].map({'R': 0, 'U': 1})
X['famsize'] = X['famsize'].map({'LE3': 0, 'GT3': 1})
X['Pstatus'] = X['Pstatus'].map({'A': 0, 'T': 1})
X['reason'] = X['reason'].map({'other':0, 'home':1, 'reputation':2, 'course':3})
X['Mjob'] = X['Mjob'].map({'other':0, 'health':1, 'services':2, 'at_home':3, 'teacher':4})
X['Fjob'] = X['Fjob'].map({'other':0, 'health':1, 'services':2, 'at_home':3, 'teacher':4})
X['guardian'] = X['guardian'].map({'other':0, 'mother':1, 'father':2})
X['schoolsup'] = X['schoolsup'].map({'no': 0, 'yes': 1})
X['famsup'] = X['famsup'].map({'no': 0, 'yes': 1})
X['paid'] = X['paid'].map({'no': 0, 'yes': 1})
X['activities'] = X['activities'].map({'no': 0, 'yes': 1})
X['nursery'] = X['nursery'].map({'no': 0, 'yes': 1})
X['higher'] = X['higher'].map({'no': 0, 'yes': 1})
X['internet'] = X['internet'].map({'no': 0, 'yes': 1})
X['romantic'] = X['romantic'].map({'no': 0, 'yes': 1})

In [5]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

## Setting ActionSet base parameters

In [6]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['school', 'sex', 'age']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = -1
    if feat.name in ['Mjob', 'Fjob']:
        feat.mutable = False
        #feat.flip_direction = 1
        feat.step_direction = 1
    if feat.name in ['famsize', 'traveltime', 'failures',
                    'romantic', 'Dalc', 'Walc', 'absences',
                     'goout', 'Pstatus', 'reason', 'activities',
                    'schoolsup',]:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = -1
    if feat.name in ['address', 'Medu',
                     'Fedu', 'guardian',
                    'studytime', 'schoolsup', 'famsup',
                     'paid', 'nursery',
                     'higher', 'internet', 'famrel',
                    'freetime', 'health']:
        feat.mutable = True
        #feat.flip_direction = -1
        feat.step_direction = 1
    feat.step_type = 'absolute'
    if feat.name in ['absences']:
        feat.step_type = 'relative'
        feat.step_size = 0.1
    feat.step_size = 1
    feat.update_grid()

## Logistic regression model

In [7]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

In [8]:
threshold = 0.5

In [9]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=threshold)
coefficients = clf_lgr['clf'].coef_[0]/clf_lgr['std'].scale_
intercept = clf_lgr['clf'].intercept_[0]-coefficients@clf_lgr['std'].mean_

/home/marcos/Documents/Experiments/envs/mapocam/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/marcos/Documents/Experiments/envs/mapocam/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


## Preparing counterfactual parameters

In [10]:
action_set = copy.deepcopy(action_set_base)

print('ActionSet stats')
print('Number of actionable features:', sum([action.actionable for action in action_set]))
print('Mean number of actions per feature:', np.nanmean([len(action._grid) if action.actionable else np.nan for action in action_set]))
print('Max number of actions per feature:', np.nanmax([len(action._grid) if action.actionable else np.nan for action in action_set]))

ActionSet stats
Number of actionable features: 25
Mean number of actions per feature: 3.24
Max number of actions per feature: 5.0


In [11]:
scores = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [12]:
percCalc = PercentileCalculator(action_set=action_set)

## Linear programming

In [13]:
lp_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActions(action_set, individual, coefficients, intercept, threshold)
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_lgr_results[i] = {}
    lp_lgr_results[i]['enum'] = lp
    lp_lgr_results[i]['indiv'] = individual
    lp_lgr_results[i]['solution'] = [int(s) if act.variable_type=='int' else float(s) for s, act in zip(lp.solution, action_set)]
    lp_lgr_results[i]['stats'] = lp.stats
    lp_lgr_results[i]['recourse'] = lp.recourse
    lp_lgr_results[i]['time'] = lp_time
    lp_lgr_results[i]['cost'] = criteria.f(lp_lgr_results[i]['solution']) if ~any(np.isnan(lp_lgr_results[i]['solution'])) else float('inf')

100%|██████████| 27/27 [00:01<00:00, 18.04it/s]


## Greedy approach

In [14]:
greedy_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    greedy = Greedy(action_set, individual, clf_lgr_mapocam, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_lgr_results[i] = {}
    greedy_lgr_results[i]['solution'] = greedy.solution
    greedy_lgr_results[i]['time'] = greedy_time
    greedy_lgr_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 27/27 [00:01<00:00, 15.09it/s]


## MAPOCAM

In [15]:
mapocam_lgr_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_lgr_results[i] = {}
    mapocam_lgr_results[i]['enum'] = mapocam
    mapocam_lgr_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_lgr_results[i]['time'] = mapocam_time
    mapocam_lgr_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 27/27 [00:04<00:00,  6.41it/s]


## Performance comparison for single objective


In [16]:
lp_lgr_costs = np.array([lp_lgr_results[i]['cost'] for i in denied_individuals])
greedy_lgr_costs = np.array([greedy_lgr_results[i]['cost'] for i in denied_individuals])
mapocam_lgr_costs = np.array([mapocam_lgr_results[i]['cost'] for i in denied_individuals])

lp_lgr_time = np.array([lp_lgr_results[i]['time'] for i in denied_individuals])
greedy_lgr_time = np.array([greedy_lgr_results[i]['time'] for i in denied_individuals])
mapocam_lgr_time = np.array([mapocam_lgr_results[i]['time'] for i in denied_individuals])

best_costs = np.min([lp_lgr_costs, greedy_lgr_costs, mapocam_lgr_costs], axis=0)

In [17]:
lgr_dict = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_lgr_time)),
                'greedy': float(np.mean(greedy_lgr_time)),
                'mi-logistic': float(np.mean(lp_lgr_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_lgr_costs==best_costs)),
               'greedy': float(np.mean(greedy_lgr_costs==best_costs)),
               'mi-logistic': float(np.mean(lp_lgr_costs==best_costs)),
           },
            'Feasible rate':1-float(np.mean(np.isnan(mapocam_lgr_costs)))
    
}
save_result(lgr_dict, 'student', 'logistic', 'MPC cost')

Best solution rate:
  MAPOCAM: 1.0
  greedy: 0.4444444444444444
  mi-logistic: 1.0
Feasible rate: 1.0
Time performance:
  MAPOCAM: 0.15505443343944433
  greedy: 0.0650258669629693
  mi-logistic: 0.05259914432773022



## Comparing Pareto enumeration

## Percentile vs changes

In [18]:
mapocam_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual,
                      clf_lgr_mapocam, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_pc[i] = {}
    mapocam_results_pc[i]['enum'] = mapocam
    mapocam_results_pc[i]['time'] = mapocam_time
    mapocam_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 27/27 [00:12<00:00,  2.14it/s]


In [19]:
lp_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_number_actions',
                                    compare=PercentileChangesCriterion(individual, percCalc))
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_pc[i] = {}
    lp_results_pc[i]['enum'] = lp
    lp_results_pc[i]['time'] = lp_time
    lp_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 27/27 [00:01<00:00, 25.76it/s]


In [20]:
bf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_pc[i] = {}
    bf_results_pc[i]['enum'] = bf
    bf_results_pc[i]['time'] = bf_time
    bf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 27/27 [01:30<00:00,  3.37s/it]


In [21]:
brute_sols_pc = np.array([bf_results_pc[i]['num'] for i in denied_individuals])
mapocam_sols_pc = np.array([mapocam_results_pc[i]['num'] for i in denied_individuals])
lp_sols_pc = np.array([lp_results_pc[i]['num'] for i in denied_individuals])
best_sols_pc = brute_sols_pc

brute_time_pc = np.array([bf_results_pc[i]['time'] for i in denied_individuals])
mapocam_time_pc = np.array([mapocam_results_pc[i]['time'] for i in denied_individuals])
lp_time_pc = np.array([lp_results_pc[i]['time'] for i in denied_individuals])

In [22]:
lgr_dict_pc = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_pc)),
                'brute-force': float(np.mean(brute_time_pc)),
                'po-mi-logistic': float(np.mean(lp_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_pc==best_sols_pc)),
               'brute-force': float(np.mean(brute_sols_pc==best_sols_pc)),
               'po-mi-logistic': float(np.mean(lp_sols_pc==best_sols_pc)),
           }
    
}
save_result(lgr_dict_pc, 'student', 'logistic', 'MPC vs. #changes')

Best solution rate:
  MAPOCAM: 1.0
  brute-force: 1.0
  po-mi-logistic: 1.0
Time performance:
  MAPOCAM: 0.46618490603631707
  brute-force: 3.3647599511659116
  po-mi-logistic: 0.03798467962554208



## Features

In [23]:
mapocam_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam = MAPOCAM(action_set, individual, clf_lgr_mapocam, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_results_feat[i] = {}
    mapocam_results_feat[i]['enum'] = mapocam
    mapocam_results_feat[i]['time'] = mapocam_time
    mapocam_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 27/27 [00:09<00:00,  2.86it/s]


In [24]:
lp_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = LinearRecourseActionsMulti(action_set, individual, coefficients, intercept,
                                    threshold, max_changes=3, enumeration_type='remove_dominated')
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_results_feat[i] = {}
    lp_results_feat[i]['enum'] = lp
    lp_results_feat[i]['time'] = lp_time
    lp_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 27/27 [00:35<00:00,  1.31s/it]


In [25]:
bf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    bf = BruteForce(action_set, individual, clf_lgr_mapocam, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_results_feat[i] = {}
    bf_results_feat[i]['enum'] = bf
    bf_results_feat[i]['time'] = bf_time
    bf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 27/27 [00:47<00:00,  1.76s/it]


In [26]:
brute_sols_feat = np.array([bf_results_feat[i]['num'] for i in denied_individuals])
mapocam_sols_feat = np.array([mapocam_results_feat[i]['num'] for i in denied_individuals])
lp_sols_feat = np.array([lp_results_feat[i]['num'] for i in denied_individuals])
best_sols_feat = brute_sols_feat

brute_time_feat = np.array([bf_results_feat[i]['time'] for i in denied_individuals])
mapocam_time_feat = np.array([mapocam_results_feat[i]['time'] for i in denied_individuals])
lp_time_feat = np.array([lp_results_feat[i]['time'] for i in denied_individuals])

In [27]:
lgr_dict_feat = {'Time performance':
            {
                'MAPOCAM': float(np.mean(mapocam_time_feat)),
                'brute-force': float(np.mean(brute_time_feat)),
                'po-mi-logistic': float(np.mean(lp_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM':float(np.mean(mapocam_sols_feat==best_sols_feat)),
               'brute-force': float(np.mean(brute_sols_feat==best_sols_feat)),
               'po-mi-logistic': float(np.mean(lp_sols_feat==best_sols_feat)),
           }
    
}
save_result(lgr_dict_feat, 'student', 'logistic', 'feature')

Best solution rate:
  MAPOCAM: 1.0
  brute-force: 1.0
  po-mi-logistic: 1.0
Time performance:
  MAPOCAM: 0.34900142169229825
  brute-force: 1.759753108723089
  po-mi-logistic: 1.309883711225767



## Random forest

In [28]:
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

In [29]:
clf_rt = ensemble.RandomForestClassifier(n_estimators=5, max_depth=5, max_leaf_nodes=31,
                                         class_weight='balanced_subsample')
clf_rt.fit(Xtr, ytr);

## Preparing counterfactual parameters

In [30]:
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction
    
print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_tree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_tree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_tree]))

ActionSet stats
Number of actionable features: 25
Mean number of actions per feature: 2.2
Max number of actions per feature: 7


In [31]:
scores = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < .8].index

In [32]:
percCalc = PercentileCalculator(action_set=action_set_tree)

In [33]:
mapocam_clf_rf = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold)
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)

## Linear programming

In [34]:
lp_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt,
                               threshold=threshold, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_rf_results[i] = {}
    lp_rf_results[i]['enum'] = lp
    lp_rf_results[i]['indiv'] = individual
    lp_rf_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_rf_results[i]['time'] = lp_time
    lp_rf_results[i]['cost'] = criteria.f(lp_rf_results[i]['solution']) if not any(np.isnan(lp_rf_results[i]['solution'])) else float('inf')

  9%|▉         | 8/87 [00:00<00:03, 22.22it/s]

gurobi version 9.0 found


100%|██████████| 87/87 [00:01<00:00, 51.98it/s]


## Greedy approach

In [35]:
greedy_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    greedy = Greedy(action_set_tree, individual, mapocam_clf_rf_fast, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_rf_results[i] = {}
    greedy_rf_results[i]['solution'] = greedy.solution
    greedy_rf_results[i]['time'] = greedy_time
    greedy_rf_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 87/87 [00:00<00:00, 126.98it/s]


## MAPOCAM

In [36]:
mapocam_rf_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results[i] = {}
    mapocam_rf_results[i]['enum'] = mapocam
    mapocam_rf_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results[i]['time'] = mapocam_time
    mapocam_rf_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 87/87 [00:36<00:00,  2.41it/s]


In [37]:
mapocam_rf_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_plus[i] = {}
    mapocam_rf_results_plus[i]['enum'] = mapocam
    mapocam_rf_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_rf_results_plus[i]['time'] = mapocam_time
    mapocam_rf_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 87/87 [00:05<00:00, 16.42it/s]


In [38]:
lp_rf_costs = np.array([lp_rf_results[i]['cost'] for i in denied_individuals])
greedy_rf_costs = np.array([greedy_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs = np.array([mapocam_rf_results[i]['cost'] for i in denied_individuals])
mapocam_rf_costs_plus = np.array([mapocam_rf_results_plus[i]['cost'] for i in denied_individuals])
best_costs_rf = np.min([lp_rf_costs, greedy_rf_costs, mapocam_rf_costs, mapocam_rf_costs_plus], axis=0)

lp_rf_time = np.array([lp_rf_results[i]['time'] for i in denied_individuals])
greedy_rf_time = np.array([greedy_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time = np.array([mapocam_rf_results[i]['time'] for i in denied_individuals])
mapocam_rf_time_plus = np.array([mapocam_rf_results_plus[i]['time'] for i in denied_individuals])

In [39]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time)),
                'greedy': float(np.mean(greedy_rf_time)),
                'mi-trees': float(np.mean(lp_rf_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_costs_plus==best_costs_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_costs==best_costs_rf)),
               'greedy': float(np.mean(greedy_rf_costs==best_costs_rf)),
               'mi-trees': float(np.mean(lp_rf_costs==best_costs_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isnan(mapocam_rf_costs_plus)))
    
}
save_result(tree_dict, 'student', 'tree', 'MPC cost')

Best solution rate:
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  greedy: 0.22988505747126436
  mi-trees: 0.9770114942528736
Feasible rate: 1.0
Time performance:
  MAPOCAM-N: 0.41435236836804995
  MAPOCAM-P: 0.060629769976400696
  greedy: 0.007651465022989988
  mi-trees: 0.018600418095894414



## Comparing Pareto enumeration

## Percentile vs changes

In [40]:
mapocam_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc[i] = {}
    mapocam_rf_results_pc[i]['enum'] = mapocam
    mapocam_rf_results_pc[i]['time'] = mapocam_time
    mapocam_rf_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 87/87 [00:09<00:00,  9.25it/s]


In [41]:
mapocam_rf_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual,
                      mapocam_clf_rf_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_pc_plus[i] = {}
    mapocam_rf_results_pc_plus[i]['enum'] = mapocam
    mapocam_rf_results_pc_plus[i]['time'] = mapocam_time
    mapocam_rf_results_pc_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 87/87 [00:04<00:00, 19.85it/s]


In [42]:
lp_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear', compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_pc[i] = {}
    lp_rf_results_pc[i]['enum'] = lp
    lp_rf_results_pc[i]['time'] = lp_time
    lp_rf_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 87/87 [00:03<00:00, 25.13it/s]


In [43]:
bf_rf_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_pc[i] = {}
    bf_rf_results_pc[i]['enum'] = bf
    bf_rf_results_pc[i]['time'] = bf_time
    bf_rf_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 87/87 [01:58<00:00,  1.37s/it]


In [44]:
lp_rf_sols_pc = np.array([lp_rf_results_pc[i]['num'] for i in denied_individuals])
brute_rf_sols_pc = np.array([bf_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc = np.array([mapocam_rf_results_pc[i]['num'] for i in denied_individuals])
mapocam_rf_sols_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['num'] for i in denied_individuals])
best_rf_sols_pc = brute_rf_sols_pc

lp_rf_time_pc = np.array([lp_rf_results_pc[i]['time'] for i in denied_individuals])
brute_rf_time_pc = np.array([bf_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc = np.array([mapocam_rf_results_pc[i]['time'] for i in denied_individuals])
mapocam_rf_time_pc_plus = np.array([mapocam_rf_results_pc_plus[i]['time'] for i in denied_individuals])

In [45]:
tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_pc)),
                'brute-force': float(np.mean(brute_rf_time_pc)),
                'po-mi-trees': float(np.mean(lp_rf_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_pc_plus==best_rf_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_pc==best_rf_sols_pc)),
               'brute-force': float(np.mean(brute_rf_sols_pc==best_rf_sols_pc)),
               'po-mi-trees': float(np.mean(lp_rf_sols_pc==best_rf_sols_pc)),
           }
    
}
save_result(tree_dict_pc, 'student', 'tree', 'MPC vs. #changes')

Best solution rate:
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  brute-force: 1.0
  po-mi-trees: 0.9885057471264368
Time performance:
  MAPOCAM-N: 0.10780480630754696
  MAPOCAM-P: 0.04998153864657613
  brute-force: 1.3645400587314118
  po-mi-trees: 0.03933843194464243



## Features

In [46]:
mapocam_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat[i] = {}
    mapocam_rf_results_feat[i]['enum'] = mapocam
    mapocam_rf_results_feat[i]['time'] = mapocam_time
    mapocam_rf_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 87/87 [00:05<00:00, 15.31it/s]


In [47]:
mapocam_clf_rf_fast = TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)

In [48]:
mapocam_rf_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    mapocam = MAPOCAM(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_rf_results_feat_plus[i] = {}
    mapocam_rf_results_feat_plus[i]['enum'] = mapocam
    mapocam_rf_results_feat_plus[i]['time'] = mapocam_time
    mapocam_rf_results_feat_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 87/87 [00:01<00:00, 44.60it/s] 


In [49]:
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])
print('Proposed:', np.mean(mapocam_rf_time_feat_plus))

Proposed: 0.02218947953533852


In [50]:
lp_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_tree, individual, clf_rt, threshold=threshold,
                               max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_rf_results_feat[i] = {}
    lp_rf_results_feat[i]['enum'] = lp
    lp_rf_results_feat[i]['time'] = lp_time
    lp_rf_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 87/87 [00:02<00:00, 38.97it/s]


In [51]:
bf_rf_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_rf_fast.fit(individual, action_set_tree)
    bf = BruteForce(action_set_tree, individual, mapocam_clf_rf_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_rf_results_feat[i] = {}
    bf_rf_results_feat[i]['enum'] = bf
    bf_rf_results_feat[i]['time'] = bf_time
    bf_rf_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 87/87 [00:31<00:00,  2.73it/s]


In [52]:
brute_rf_sols_feat = np.array([bf_rf_results_feat[i]['num'] for i in denied_individuals])
lp_rf_sols_feat = np.array([lp_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat = np.array([mapocam_rf_results_feat[i]['num'] for i in denied_individuals])
mapocam_rf_sols_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['num'] for i in denied_individuals])
best_rf_sols_feat = brute_rf_sols_feat

brute_rf_time_feat = np.array([bf_rf_results_feat[i]['time'] for i in denied_individuals])
lp_rf_time_feat = np.array([lp_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat = np.array([mapocam_rf_results_feat[i]['time'] for i in denied_individuals])
mapocam_rf_time_feat_plus = np.array([mapocam_rf_results_feat_plus[i]['time'] for i in denied_individuals])

In [53]:
tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-P': float(np.mean(mapocam_rf_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_rf_time_feat)),
                'brute-force': float(np.mean(brute_rf_time_feat)),
                'po-mi-trees': float(np.mean(lp_rf_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-P':float(np.mean(mapocam_rf_sols_feat_plus==best_rf_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_rf_sols_feat==best_rf_sols_feat)),
               'brute-force': float(np.mean(brute_rf_sols_feat==best_rf_sols_feat)),
               'po-mi-trees': float(np.mean(lp_rf_sols_feat==best_rf_sols_feat)),
           }
    
}
save_result(tree_dict_feat, 'student', 'tree', 'feature')

Best solution rate:
  MAPOCAM-N: 1.0
  MAPOCAM-P: 0.9885057471264368
  brute-force: 1.0
  po-mi-trees: 0.9655172413793104
Time performance:
  MAPOCAM-N: 0.06492329913930132
  MAPOCAM-P: 0.02218947953533852
  brute-force: 0.3648677436091627
  po-mi-trees: 0.025277780211027497



## Monotone trees

In [54]:
from sklearn import tree, ensemble
import lightgbm
from cfmining.predictors import MonotoneTree

In [55]:
monotone_constraints = [action.step_direction for action in action_set_base]
clf_mt = lightgbm.LGBMClassifier(n_estimators=10, max_depth=10, num_leaves=63,
                                 monotone_constraints=monotone_constraints, class_weight='balanced')
clf_mt.fit(Xtr, ytr)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=10, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0,
               monotone_constraints=[-1, -1, -1, 1, -1, -1, 1, 1, 1, 1, -1, 1,
                                     -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1,
                                     -1, -1, -1, 1, -1],
               n_estimators=10, n_jobs=-1, num_leaves=63, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

## Preparing counterfactual parameters

In [56]:
action_set_mtree = copy.deepcopy(action_set_base)
action_set_mtree.embed_forest(clf_mt)
for action in action_set_mtree:
    action.flip_direction = action.step_direction

print('ActionSet stats')
print('Number of actionable features:', sum([action.mutable for action in action_set_mtree]))
print('Mean number of actions per feature:', np.mean([len(action._grid)-1 for action in action_set_mtree]))
print('Max number of actions per feature:', np.max([len(action._grid)-1 for action in action_set_mtree]))

ActionSet stats
Number of actionable features: 25
Mean number of actions per feature: 1.5
Max number of actions per feature: 3


In [57]:
scores = pd.Series(clf_mt.predict_proba(Xts)[:, 1])
denied_individuals = scores.loc[lambda s: s < threshold].index

In [58]:
percCalc = PercentileCalculator(action_set=action_set_mtree)

In [59]:
mapocam_clf_mt = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold)
mapocam_clf_mt_fast = TreeClassifier(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)
mapocam_clf_mt_mon = MonotoneTree(clf_mt, Xtr, ytr, clf_type='lightgbm', threshold=threshold, use_predict_max=True)

## Linear programming

In [60]:
lp_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm',
                               threshold=threshold, max_changes=float('inf'))
    lp.build_model()
    lp.fit(threads=1)
    lp_time = time.perf_counter()-start
    
    lp_mt_results[i] = {}
    lp_mt_results[i]['enum'] = lp
    lp_mt_results[i]['indiv'] = individual
    lp_mt_results[i]['solution'] = lp.solution#[int(s) if act.variable_type=='int' else float(s) for s, act in zip(pl.solution, action_set)]
    lp_mt_results[i]['time'] = lp_time
    lp_mt_results[i]['cost'] = criteria.f(lp_mt_results[i]['solution']) if not any(np.isnan(lp_mt_results[i]['solution'])) else float('inf')

100%|██████████| 32/32 [00:00<00:00, 47.83it/s]


## Greedy approach

In [61]:
greedy_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    greedy = Greedy(action_set_mtree, individual, mapocam_clf_mt, criteria)
    greedy.fit()
    greedy_time = time.perf_counter()-start
    
    greedy_mt_results[i] = {}
    greedy_mt_results[i]['solution'] = greedy.solution
    greedy_mt_results[i]['time'] = greedy_time
    greedy_mt_results[i]['cost'] = criteria.f(greedy.solution) if greedy.solution is not None else float('inf')

100%|██████████| 32/32 [00:00<00:00, 81.47it/s]


## MAPOCAME

In [62]:
mapocam_mt_results = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results[i] = {}
    mapocam_mt_results[i]['enum'] = mapocam
    mapocam_mt_results[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results[i]['time'] = mapocam_time
    mapocam_mt_results[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 32/32 [00:01<00:00, 22.27it/s]


In [63]:
mapocam_mt_results_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_plus[i] = {}
    mapocam_mt_results_plus[i]['enum'] = mapocam
    mapocam_mt_results_plus[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_plus[i]['time'] = mapocam_time
    mapocam_mt_results_plus[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 32/32 [00:00<00:00, 56.83it/s]


In [64]:
mapocam_mt_results_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=float('inf'), compare=criteria)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_mon[i] = {}
    mapocam_mt_results_mon[i]['enum'] = mapocam
    mapocam_mt_results_mon[i]['solution'] = mapocam.solutions[0] if len(mapocam.solutions)>0 else None
    mapocam_mt_results_mon[i]['time'] = mapocam_time
    mapocam_mt_results_mon[i]['cost'] = criteria.f(mapocam.solutions[0])  if len(mapocam.solutions)>0 else float('inf')

100%|██████████| 32/32 [00:00<00:00, 87.66it/s]


In [65]:
lp_mt_costs = np.array([lp_mt_results[i]['cost'] for i in denied_individuals])
greedy_mt_costs = np.array([greedy_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs = np.array([mapocam_mt_results[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_plus = np.array([mapocam_mt_results_plus[i]['cost'] for i in denied_individuals])
mapocam_mt_costs_mon = np.array([mapocam_mt_results_mon[i]['cost'] for i in denied_individuals])
best_costs_mt_rf = np.min([lp_mt_costs, greedy_mt_costs, mapocam_mt_costs, mapocam_mt_costs_plus, mapocam_mt_costs_mon], axis=0)

lp_mt_time = np.array([lp_mt_results[i]['time'] for i in denied_individuals])
greedy_mt_time = np.array([greedy_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time = np.array([mapocam_mt_results[i]['time'] for i in denied_individuals])
mapocam_mt_time_plus = np.array([mapocam_mt_results_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_mon = np.array([mapocam_mt_results_mon[i]['time'] for i in denied_individuals])

In [66]:
tree_dict = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time)),
                'greedy': float(np.mean(greedy_mt_time)),
                'mi-trees': float(np.mean(lp_mt_time)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_costs_mon==best_costs_mt_rf)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_costs_plus==best_costs_mt_rf)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_costs==best_costs_mt_rf)),
               'greedy': float(np.mean(greedy_mt_costs==best_costs_mt_rf)),
               'mi-trees': float(np.mean(lp_mt_costs==best_costs_mt_rf)),
           },
             'Feasible rate':1-float(np.mean(np.isnan(mapocam_mt_costs_mon)))
    
}
save_result(tree_dict, 'student', 'monotone-tree', 'MPC cost')

Best solution rate:
  MAPOCAM-M: 0.71875
  MAPOCAM-N: 0.71875
  MAPOCAM-P: 0.71875
  greedy: 0.625
  mi-trees: 1.0
Feasible rate: 1.0
Time performance:
  MAPOCAM-M: 0.011156133932672674
  MAPOCAM-N: 0.044391121595253935
  MAPOCAM-P: 0.01728265065685264
  greedy: 0.01202316147464444
  mi-trees: 0.019985057369922288



## Comparing Pareto enumeration

## Percentile vs changes

In [67]:
mapocam_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc[i] = {}
    mapocam_mt_results_pc[i]['enum'] = mapocam
    mapocam_mt_results_pc[i]['time'] = mapocam_time
    mapocam_mt_results_pc[i]['num'] = len(mapocam.solutions)

100%|██████████| 32/32 [00:06<00:00,  5.12it/s]


In [68]:
mapocam_mt_results_pc_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_fast, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_plus[i] = {}
    mapocam_mt_results_pc_plus[i]['enum'] = mapocam
    mapocam_mt_results_pc_plus[i]['time'] = mapocam_time
    mapocam_mt_results_pc_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 32/32 [00:00<00:00, 32.25it/s]


In [69]:
mapocam_mt_results_pc_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual,
                      mapocam_clf_mt_mon, max_changes=3,
                      compare=PercentileChangesCriterion(individual, percCalc),
                      recursive=False)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_pc_mon[i] = {}
    mapocam_mt_results_pc_mon[i]['enum'] = mapocam
    mapocam_mt_results_pc_mon[i]['time'] = mapocam_time
    mapocam_mt_results_pc_mon[i]['num'] = len(mapocam.solutions)

100%|██████████| 32/32 [00:00<00:00, 55.53it/s]


In [70]:
lp_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm',
                               threshold=threshold, max_changes=3, multi_solution=True,
                               cost_type='linear',
                               compare=PercentileChangesCriterion(individual, percCalc))
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_pc[i] = {}
    lp_mt_results_pc[i]['enum'] = lp
    lp_mt_results_pc[i]['time'] = lp_time
    lp_mt_results_pc[i]['num'] = len(lp.solutions)

100%|██████████| 32/32 [00:01<00:00, 23.20it/s]


In [71]:
bf_mt_results_pc = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, compare=PercentileChangesCriterion(individual, percCalc))
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_pc[i] = {}
    bf_mt_results_pc[i]['enum'] = bf
    bf_mt_results_pc[i]['time'] = bf_time
    bf_mt_results_pc[i]['num'] = len(bf.solutions)

100%|██████████| 32/32 [00:17<00:00,  1.80it/s]


In [72]:
brute_mt_sols_pc = np.array([bf_mt_results_pc[i]['num'] for i in denied_individuals])
lp_mt_sols_pc = np.array([lp_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc = np.array([mapocam_mt_results_pc[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['num'] for i in denied_individuals])
best_mt_sols_pc = brute_mt_sols_pc

brute_mt_time_pc = np.array([bf_mt_results_pc[i]['time'] for i in denied_individuals])
lp_mt_time_pc = np.array([lp_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc = np.array([mapocam_mt_results_pc[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_plus = np.array([mapocam_mt_results_pc_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_pc_mon = np.array([mapocam_mt_results_pc_mon[i]['time'] for i in denied_individuals])

In [73]:
mon_tree_dict_pc = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_pc_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_pc_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_pc)),
                'brute-force': float(np.mean(brute_mt_time_pc)),
                'po-mi-trees': float(np.mean(lp_mt_time_pc)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc_mon)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_pc_plus==best_mt_sols_pc)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_pc==best_mt_sols_pc)),
               'brute-force': float(np.mean(brute_mt_sols_pc==best_mt_sols_pc)),
               'po-mi-trees': float(np.mean(lp_mt_sols_pc==best_mt_sols_pc)),
           }
    
}
save_result(mon_tree_dict_pc, 'student', 'monotone-tree', 'MPC vs. #changes')

Best solution rate:
  MAPOCAM-M: 1.0
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  brute-force: 1.0
  po-mi-trees: 0.84375
Time performance:
  MAPOCAM-M: 0.017804400256864028
  MAPOCAM-N: 0.1948893819717341
  MAPOCAM-P: 0.030763689035666175
  brute-force: 0.553160309777013
  po-mi-trees: 0.04270600018571713



## Features

In [74]:
mapocam_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat[i] = {}
    mapocam_mt_results_feat[i]['enum'] = mapocam
    mapocam_mt_results_feat[i]['time'] = mapocam_time
    mapocam_mt_results_feat[i]['num'] = len(mapocam.solutions)

100%|██████████| 32/32 [00:07<00:00,  4.43it/s]


In [75]:
mapocam_mt_results_feat_plus = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_plus[i] = {}
    mapocam_mt_results_feat_plus[i]['enum'] = mapocam
    mapocam_mt_results_feat_plus[i]['time'] = mapocam_time
    mapocam_mt_results_feat_plus[i]['num'] = len(mapocam.solutions)

100%|██████████| 32/32 [00:02<00:00, 14.31it/s]


In [76]:
mapocam_mt_results_feat_mon = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_mon.fit(individual, action_set_mtree)
    mapocam = MAPOCAM(action_set_mtree, individual, mapocam_clf_mt_mon, max_changes=3, recursive=True)
    mapocam.fit()
    mapocam_time = time.perf_counter()-start
    
    mapocam_mt_results_feat_mon[i] = {}
    mapocam_mt_results_feat_mon[i]['enum'] = mapocam
    mapocam_mt_results_feat_mon[i]['time'] = mapocam_time
    mapocam_mt_results_feat_mon[i]['num'] = len(mapocam.solutions)

100%|██████████| 32/32 [00:00<00:00, 37.76it/s]


In [77]:
lp_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc)
    
    start = time.perf_counter()
    lp = ForestRecourseActions(action_set_mtree, individual, clf_mt, clf_type='lightgbm',
                               threshold=threshold, max_changes=3, multi_solution=True,
                               cost_type='linear')
    lp.build_model()
    lp.fit()
    lp_time = time.perf_counter()-start
    
    lp_mt_results_feat[i] = {}
    lp_mt_results_feat[i]['enum'] = lp
    lp_mt_results_feat[i]['time'] = lp_time
    lp_mt_results_feat[i]['num'] = len(lp.solutions)

100%|██████████| 32/32 [00:01<00:00, 19.14it/s]


In [78]:
bf_mt_results_feat = {}
for i in tqdm(denied_individuals):
    individual = Xts.iloc[i].values
    
    start = time.perf_counter()
    mapocam_clf_mt_fast.fit(individual, action_set_mtree)
    bf = BruteForce(action_set_mtree, individual, mapocam_clf_mt_fast, max_changes=3)
    bf.fit()
    bf_time = time.perf_counter()-start
    
    bf_mt_results_feat[i] = {}
    bf_mt_results_feat[i]['enum'] = bf
    bf_mt_results_feat[i]['time'] = bf_time
    bf_mt_results_feat[i]['num'] = len(bf.solutions)

100%|██████████| 32/32 [00:08<00:00,  3.67it/s]


In [79]:
brute_mt_sols_feat = np.array([bf_mt_results_feat[i]['num'] for i in denied_individuals])
lp_mt_sols_feat = np.array([lp_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat = np.array([mapocam_mt_results_feat[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['num'] for i in denied_individuals])
mapocam_mt_sols_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['num'] for i in denied_individuals])
best_mt_sols_feat = brute_mt_sols_feat

brute_mt_time_feat = np.array([bf_mt_results_feat[i]['time'] for i in denied_individuals])
lp_mt_time_feat = np.array([lp_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat = np.array([mapocam_mt_results_feat[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_plus = np.array([mapocam_mt_results_feat_plus[i]['time'] for i in denied_individuals])
mapocam_mt_time_feat_mon = np.array([mapocam_mt_results_feat_mon[i]['time'] for i in denied_individuals])

In [80]:
print('Changes vs percentile:', np.mean(brute_mt_sols_pc==mapocam_mt_sols_pc)==1, np.mean(lp_mt_sols_pc==mapocam_mt_sols_pc)==1, np.mean(mapocam_mt_sols_pc_plus==mapocam_mt_sols_pc)==1)
print('Features:', np.mean(brute_mt_sols_feat==mapocam_mt_sols_feat)==1, np.mean(lp_mt_sols_feat==mapocam_mt_sols_feat)==1, np.mean(mapocam_mt_sols_feat_plus==mapocam_mt_sols_feat)==1)

Changes vs percentile: True False True
Features: True False True


In [81]:
mon_tree_dict_feat = {'Time performance':
            {
                'MAPOCAM-M': float(np.mean(mapocam_mt_time_feat_mon)),
                'MAPOCAM-P': float(np.mean(mapocam_mt_time_feat_plus)),
                'MAPOCAM-N': float(np.mean(mapocam_mt_time_feat)),
                'brute-force': float(np.mean(brute_mt_time_feat)),
                'po-mi-trees': float(np.mean(lp_mt_time_feat)),
            },
            'Best solution rate':
           {
               'MAPOCAM-M':float(np.mean(mapocam_mt_sols_feat_mon==best_mt_sols_feat)),
               'MAPOCAM-P':float(np.mean(mapocam_mt_sols_feat_plus==best_mt_sols_feat)),
               'MAPOCAM-N':float(np.mean(mapocam_mt_sols_feat==best_mt_sols_feat)),
               'brute-force': float(np.mean(brute_mt_sols_feat==best_mt_sols_feat)),
               'po-mi-trees': float(np.mean(lp_mt_sols_feat==best_mt_sols_feat)),
           }
    
}
save_result(mon_tree_dict_feat, 'student', 'monotone-tree', 'feature')

Best solution rate:
  MAPOCAM-M: 1.0
  MAPOCAM-N: 1.0
  MAPOCAM-P: 1.0
  brute-force: 1.0
  po-mi-trees: 0.3125
Time performance:
  MAPOCAM-M: 0.02603530290798517
  MAPOCAM-N: 0.22476645882125013
  MAPOCAM-P: 0.06919568255761988
  brute-force: 0.27135760197779746
  po-mi-trees: 0.05166537326294929

